# PLANTEAMIENTOS DE PROBLEMAS

## Ejercicios con Julia

Una vez que se ha instalado el solver correspondiente **CPLEX** o bien **Gurobi**, podemos comenzar a construir los modelos para los problemas a optimizar.
Primero vamos a ver un ejemplo:


\begin{equation*}
\begin{array}{lll}
    \mbox{Maximizar}
    & x_1+2x_2+5x_3 &\\
    \mbox{sujeto a: }
    & -x_1+x_2+3x_3 \leq -5\\
    & x_1 +3x_2 -7x_3 \leq 10\\
    & 0 \leq x_1 \leq 10\\
    & x_2, x_3 \geq 0
    \end{array}
\end{equation*}

In [1]:
# paquetes necesarios para resolver el modelo
using CPLEX, JuMP

In [2]:
# Preparación del modelo a optimizar
m = Model(CPLEX.Optimizer)

# Declaración de variables
@variable(m, 0<= x1 <=10)
@variable(m, x2 >=0)
@variable(m, x3 >=0)

# Construcción de la función objetivo
@objective(m, Max, x1 + 2x2 + 5x3)

# Restricciones del problema
@constraint(m, constraint1, -x1 +  x2 + 3x3 <= -5)
@constraint(m, constraint2,  x1 + 3x2 - 7x3 <= 10)

# imprimimos el modelo construido
print(m)

# optimizador del modelo
JuMP.optimize!(m)

# Impresión de soluciones de las variables para el problema primal
println("Optimal Solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

Max x1 + 2 x2 + 5 x3
Subject to
 constraint1 : -x1 + x2 + 3 x3 <= -5.0
 constraint2 : x1 + 3 x2 - 7 x3 <= 10.0
 x1 >= 0.0
 x2 >= 0.0
 x3 >= 0.0
 x1 <= 10.0
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =            19.062500
Optimal Solutions:
x1 = 10.0
x2 = 2.1875
x3 = 0.9375


La declaración de las tres variables para este problema se hace en las líneas:

Al estar utilizando el símbolo de @, se esta haciendo uso de una especie de 'macros' implementada en el paquete JuMP.

También para la construcción de la función objetivo se utiliza la macro llamada @objetive, la cual recibe las siguientes entradas:

`@objetive(modelo,Max/Min,variables)`

Para el caso de nuestro ejemplo, tenemos dos restricciones que construimos utilizando la macros **constraint**, la cual recibe los parámetros:

`@constraint(modelo, nombre de restriccción, restriccion)`

La colocación de los nombres son recomendados cuando se manejan un número pequeño de restricciones, cuando se colocan de manera general, en esa entrada se puede colocar lo siguiente:

`[i=1:m]` donde m es el número de restricciones o bien el número de renglones de la matriz de restricciones.

Si fuera necesario checar por última vez que el modelo construido esta bien escrito, podemos imprimirlo mediante la siguiente línea.

Y finalmente que checamos que el modelo estuvo bien escrito, resolvemos el problema de optimización.

Para obtener los valores de las variables del problema se utiliza la función `JuMP.value()`:

Si se desea usar el optimizador Gurobi, entonces se utiliza la siguiente línea que inicializa un modelo de la clase Gurobi.

## Ejemplo 1.  

  El laboratorio de Wonka requiere de las materias primas I y II para producir dos soluciones de caramelo, A y B. Las disponibilidades diarias de las materias primas I y II son de 150 y 145 uniddes, respectivamente.
  
 Una unidad de solución A consume 0.5 unidades de la materia prima I y 0.6 unidades de la materia prima II, en tanto una unidad de la solución B consume 0.5 unidades de la materia prima I y 0.4 unidades de la materia prima II. Las ganancias por cada unidad vendida del caramelo A son de 8 pesos y del caramelo B son de 10 pesos. La demanda diaria de la solución A es de entre 30 y 150 unidades, y la demanda de la solución B son de 40 a 200 unidades. Determinar las cantidades de producción óptimas de A y B.

La solución del problema queda planteado como:

**Variables**:

$x_i$= unidades de la solución i (i=A,B)

**Función objetivo y restricciones:**

\begin{equation*}
\begin{array}{lll}
    \mbox{Maximizar}
    & 8x_1+10x_2 &\\
    \mbox{sujeto a: }
    & 0.5x_1+ 0.5x_2 \leq 150\\
    & 0.6x_1 + 0.4x_2 \leq 145\\
    & 30 \leq x_1 \leq 150\\
    & 40 \leq x_2 \leq 200\\
    & x_1,x_2 \geq 0
    \end{array}
\end{equation*}


### Alternativa rápida de escribir el problema 1

Para el primer ejemplo, teníamos 3 variables de decisión, lo cual podemos escribir de manera general con un vector de 3 dimensiones:

Entonces podemos ver la función objetivo como la suma producto de un vector x de variables por un vector columna de costos c.

Y las restricciones pueden verse como la multiplicación de una matriz A por el vector x de variables, esto es menor igual o mayor igual a un vector columna b que representa el vector de recursos.

Pero que sucedería si tenemos más de 20 restricciones?! se vuelve poco óptimo meter cada restricción, por lo tanto podemos generalizar haciendo lo siguiente:

Finalmente se añade la restricción que tiene la variable 1.

Se añade todo esto en un solo bloque:


O bien podemos escribirlo de la siguiente manera:

De manera general para un problema del siguiente estilo

\begin{equation*}
\begin{array}{lll}
    \mbox{Maximizar}
    & \sum_{i=1}^{n} x_i &\\
    \mbox{sujeto a: }
    & Ax \leq b \\
    & \forall x_i \geq 0; & i= 1, \ldots, n
    \end{array}
\end{equation*}